In [1]:
# Import Libaries and Tools
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import re

#plt.rcParams.update({'font.size': 30})

ROOT_DIR = os.path.abspath("../../")

In [2]:
# Collection of Results
datasetfiles = sorted(glob.glob('Results/CR*.csv.intInvStats.npz'))

print(datasetfiles)

['Results/CR2099.csv.intInvStats.npz', 'Results/CR2100.csv.intInvStats.npz', 'Results/CR2101.csv.intInvStats.npz', 'Results/CR2133.csv.intInvStats.npz']


In [3]:
# Open Data Summary
for i in range(len(datasetfiles)-1):
    
    # open and retreave data
    data = np.load(datasetfiles[i], allow_pickle=True)
    lst = data.files
    
    # Get inital list of stats
    if i == 0:
        sucess = data[lst[0]]
        alphas = data[lst[1]]
        IOU    = data[lst[2]]
        SSIM   = data[lst[3]]
        GCE    = data[lst[4]]
        LCE    = data[lst[5]]
    
    else:
        sucess = np.vstack([sucess , data[lst[0]]])
        alphas = np.vstack([alphas , data[lst[1]]])
        IOU    = np.vstack([IOU    , data[lst[2]]])
        SSIM   = np.vstack([SSIM   , data[lst[3]]])
        GCE    = np.vstack([GCE    , data[lst[4]]])
        LCE    = np.vstack([LCE    , data[lst[4]]])

In [4]:
# Formatting
ScalingTypes = ['LinearCompressFull.','LinearCompressFullRestored.','LinearCompress1toMax.',
                'LinearCompress0toMaxRestored.','LinearCompressSolarLimits.','LinearCompressSolarLimitsRestored.',
                'LinearCompressDefault.','LinearCompressDefaultRestored.','Log10CompressFull.',
                'Log10CompressFullRestored.','Log10Compress1toMax.','Log10Compress1toMaxRestored.',
                'Log10CompressSolarLimits.','Log10CompressSolarLimitsRestored.',
                'Log10CompressDefault.','Log10CompressDefaultRestored.']
# Format Keys
keysFormated = ['Linear\nFull','Linear\nFull\nRestored','Linear\n0 to Max',
                'Linear\n0 to Max\nRestored','Linear\nSolar Limits','Linear\nSolar Limits\nRestored',
                'Linear\nDefault','Linear\nDefault\nRestored','Log10\nFull','Log10\nFull\nRestored',
                'Log10\n0 to Max','Log10\n0 to Max\nRestored','Log10\nSolar Limits',
                'Log10\nSolar Limits\nRestored','Log10\nDefault','Log10\nDefault\nRestored']

In [5]:
# Determin how many times ACWE failed to generate a segmentation
Error  = np.where(sucess==0)[1]
error = np.zeros(len(keysFormated))
for e in Error:
    error[e] = error[e] + 1

# Determin how many times ACWE returned an empty segmentation
Colaps = np.where(sucess==0.5)[1]
colaps = np.zeros(len(keysFormated))
for c in Colaps:
    colaps[c] = colaps[c] + 1

# Print Results
for i in range(len(ScalingTypes)):
    print(ScalingTypes[i])
    print('    Errors:',error[i],'(',error[i]/len(sucess)*100,'%)')
    print('    Empty Seg:',colaps[i],'(',colaps[i]/len(sucess)*100,'%)')
    print()

LinearCompressFull.
    Errors: 0.0 ( 0.0 %)
    Empty Seg: 1.0 ( 0.05672149744753262 %)

LinearCompressFullRestored.
    Errors: 0.0 ( 0.0 %)
    Empty Seg: 1.0 ( 0.05672149744753262 %)

LinearCompress1toMax.
    Errors: 0.0 ( 0.0 %)
    Empty Seg: 0.0 ( 0.0 %)

LinearCompress0toMaxRestored.
    Errors: 0.0 ( 0.0 %)
    Empty Seg: 0.0 ( 0.0 %)

LinearCompressSolarLimits.
    Errors: 0.0 ( 0.0 %)
    Empty Seg: 1.0 ( 0.05672149744753262 %)

LinearCompressSolarLimitsRestored.
    Errors: 0.0 ( 0.0 %)
    Empty Seg: 0.0 ( 0.0 %)

LinearCompressDefault.
    Errors: 0.0 ( 0.0 %)
    Empty Seg: 0.0 ( 0.0 %)

LinearCompressDefaultRestored.
    Errors: 0.0 ( 0.0 %)
    Empty Seg: 0.0 ( 0.0 %)

Log10CompressFull.
    Errors: 0.0 ( 0.0 %)
    Empty Seg: 1763.0 ( 100.0 %)

Log10CompressFullRestored.
    Errors: 0.0 ( 0.0 %)
    Empty Seg: 0.0 ( 0.0 %)

Log10Compress1toMax.
    Errors: 0.0 ( 0.0 %)
    Empty Seg: 1763.0 ( 100.0 %)

Log10Compress1toMaxRestored.
    Errors: 0.0 ( 0.0 %)
    Empty S

In [6]:
# Filter

# Filter out completly empty cases 
IOUsimplified  = np.delete(IOU,10,axis=1)
IOUsimplified  = np.delete(IOUsimplified,8,axis=1)

SSIMsimplified = np.delete(SSIM,10,axis=1)
SSIMsimplified = np.delete(SSIMsimplified,8,axis=1)

GCEsimplified  = np.delete(GCE,10,axis=1)
GCEsimplified  = np.delete(GCEsimplified,8,axis=1)

LCEsimplified  = np.delete(LCE,10,axis=1)
LCEsimplified  = np.delete(LCEsimplified,10,axis=1)

# Filter Out NaNs (a NaN is present iff no segmentation is generated)
maskIOU      = ~np.isnan(IOUsimplified)
IOUfiltered  = [d[m] for d, m in zip(IOUsimplified.T, maskIOU.T)]

maskSSIM     = ~np.isnan(SSIMsimplified)
SSIMfiltered = [d[m] for d, m in zip(SSIMsimplified.T,maskSSIM.T)]

maskGCE      = ~np.isnan(GCEsimplified)
GCEfiltered  = [d[m] for d, m in zip(GCEsimplified.T, maskGCE.T)]

maskLCE      = ~np.isnan(LCEsimplified)
LCEfiltered  = [d[m] for d, m in zip(LCEsimplified.T, maskLCE.T)]

In [7]:
# Ensure Folder Exists
saveDirectory = '/Figures/IntConstancy/'
saveDirectory = ROOT_DIR + saveDirectory

if not os.path.exists(saveDirectory):
    os.makedirs(saveDirectory)

In [ ]:
# Set up lables
x = np.arange(len(keysFormated)) + 1

plt.figure(figsize=[20,7])
plt.rcParams.update({'font.size': 25})
title = 'Intersection Over Union as a function of intensity scale over CRs 2099-2101'
plt.boxplot(IOUfiltered)
plt.title(title)
#plt.xticks(x,keysFormated)
plt.grid()
title ='IOU_Box_earlyData.eps'
title = saveDirectory + title
plt.savefig(title)
plt.show()

plt.figure(figsize=[20,7])
plt.rcParams.update({'font.size': 25})
title = 'Structural Similarity as a function of intensity scale over CRs 2099-2101'
plt.boxplot(SSIMfiltered)
plt.title(title)
#plt.xticks(x,keysFormated)
plt.grid()
title='SSIM_Box_earlyData.eps'
title = saveDirectory + title
plt.savefig(title)
plt.show()

plt.figure(figsize=[20,7])
plt.rcParams.update({'font.size': 25})
title = 'Global Consistency Error as a function of intensity scale over CRs 2099-2101'
plt.boxplot(GCEfiltered)
plt.title(title)
#plt.xticks(x,keysFormated)
plt.grid()
title='GCE_Box_earlyData.eps'
title = saveDirectory + title
plt.savefig(title)
plt.show()


plt.figure(figsize=[20,7])
plt.rcParams.update({'font.size': 25})
title = 'Local Consistency Error as a function of intensity scale over CRs 2099-2101'
plt.boxplot(LCEfiltered)
plt.title(title)
#plt.xticks(x,keysFormated)
plt.grid()
title='LCE_Box_earlyData.eps'
title = saveDirectory + title
plt.savefig(title)
plt.show()